Данная программа просто перейдет по ссылкам, где архив скачается автоматически (будьте аккруратны при запуске)

In [13]:
import gc
import os
import functools
import tqdm
import shutil
import requests
import pandas as pd
import numpy as np

from pyunpack import Archive
from bs4 import BeautifulSoup
from pathlib import Path
from multiprocessing import Pool
from dbfread import DBF
from typing import Union

import re
import time

In [203]:
def load_bank_statements(form_number:int, 
                         filepath: str,
                         overwrite:bool=True) -> None:
    '''
    Loads archives with bank statements from CBR website and unpacks them into given folder
    
    Parameters
    ----------
    form_number: int, 101 or 102
            Number of CBR form of financial statements (form 101, form 102)
    filepath: str
            Directory (folder) to save .zip and .rar archives  downloaded from CBR site
            as well as unpacked .dbf files from archives
    overwrite: bool
            Whether to overwrite directories with archives and unzipped data if they already
            exist (option overwrite=False is currently not available)
    '''

    print('Downloading and unpacking files from www.cbr.ru, please be patient...')
    
    url = 'https://www.cbr.ru/banking_sector/otchetnost-kreditnykh-organizaciy/'
    

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html')

    # выберем все ссылки для нужной формы отчётности
    all_refs = [x['href'] for x in soup.find_all('a', href=True)]
    refs = ['https://cbr.ru/' + x for x in all_refs \
            if 'forms/' + str(form_number) in x]
    
    # сгенерируем кортежи аргументов для параллельной функции
    args = ((x, 
             (
             (x.split('/')[-1]).split('.')[0]
             ).split('-')[1], 
             load_path, 
             save_path,
             True) for x in refs)
    
    # параллельные действия
    #if __name__=='__main__':
       # with Pool() as pool:
         #   pool.starmap(load_and_unpack, 
                     # to show progress bar
           # tqdm.tqdm(args, total=len(refs)))        
            
    # на всякий случай сохранил альтернативный вариант с циклом
    # он работает в 4 раза медленнее, чем распараллеленный вариант

    for ref in refs:
        webbrowser.open(ref, new=0, autoraise=True)
        time.sleep(5)
        

In [205]:
load_bank_statements(102, 
                         '/Users/roman142001/Downloads') 